In [ ]:
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from time import sleep
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

pd.options.display.max_columns = 30

In [ ]:
Game = pd.read_csv('./data/GameLogs.csv',encoding='utf-16')

## Web scraping code sports-reference
[Sports Reference](https://www.sports-reference.com/) is an online sports website that stores various stats and history for major games including MLB, NBA, NFL, NHL and more. From this website we acquire baseball game/team records, performance and NBA/NFL/NHL schedules.

In [ ]:
class WebData:
  """Represents a online stored data source. Attribute: url"""

  api_key = "xxxxxxxxxxxxxxxxxx"
  proxy = {"http":"http://xxxxxxxxxxxxxxxxxx".format(api_key)}

  def __init__(self, url):
    self.url = url

  def __str__(self):
    return url

  def get_bs(self, proxies=proxy):
    """Return parsed html doc"""
    html = requests.get(self.url
                        , proxies=proxies
                        )
    bs = BeautifulSoup(html.text, 'html.parser')
    return bs


  def get_table_df(self, table_class, columns):
    """Return table dataframe.

    Arguments:
    table_class -- table class
    columns -- self-defined column names
    """
    bs = self.get_bs()
    df = pd.DataFrame(columns=columns)
    for game in bs.find('table', {'class':table_class}).find_all('tr'):
      results = []
      for element in game.find_all('td'):
        results.append(element.text)
        if len(results) == len(columns):
          df.loc[len(df)] = results
    return df

In [ ]:
########## baseball reference data processing ##########
def baseball_gamebygame_records_data_processing(df, year):
  # Problem 1. There are multiple games in one day between two teams, e.g. NYM-WSN, Oct4, 2022, at Night. How many of these records are there?

  # 1. adjust date and convert to datetime
  df.date = df.date.str.replace(r"\(.*\)","")
  df.date = df.date.apply(lambda x: x.split(', ')[1] + ', ' + str(year))
  df.date = pd.to_datetime(df.date)

  # 2. get season end rank
  df['season_end_rank'] = df.iloc[-1,:]['div_rank']
  df['season_end_rank'] = df['season_end_rank'].astype('int32')

  # 3. calculate this season W-L% and season end W-L%
  df['w_or_l_int'] = np.where(df['w_or_l'].str.startswith('W'),1,0)
  df['w_l_ratio'] = df['w_or_l_int'].cumsum()/(df['w_or_l_int'].index+1)
  df['season_end_w_l_ratio'] = df.iloc[-1,:]['w_l_ratio']

  # 4. last game performance - shift record, div_rank, gb, streak and fill in the first value
  df['last_game_record'] = df.record.shift()
  df.last_game_record.fillna('0-0', inplace=True)

  for col in ['w_l_ratio', 'div_rank', 'gb', 'streak']:
    new_col_name = 'last_game_'+col
    df[new_col_name] = df[col].shift()
    df[new_col_name].fillna('0', inplace=True)

  # 5. calculate rolling means (5) for runs and runs allowed
  df['runs_ma'] = df.runs.rolling(5).mean().shift()
  df.runs_ma.fillna(df.iloc[5].runs_ma, inplace=True)
  df['runs_allowed_ma'] = df.runs_allowed.rolling(5).mean().shift()
  df.runs_allowed_ma.fillna(df.iloc[5].runs_allowed_ma, inplace=True)

  # 6. calculate season end means for runs and runs allowed
  df['season_end_runs_mean'] = df.runs.astype('int32').mean().round(2)
  df['season_end_runs_allowed_mean'] = df.runs_allowed.astype('int32').mean().round(2)

  # 7. win_pct of last ten games
  ten_game = [list(df.w_or_l[i-10:i]).count('W') / 10. for i in range(10, len(df))]
  for i in range(10):
      ten_game.insert(0, None)
  df['last_ten_game_w_l_ratio'] = ten_game

  return df

In [ ]:
########## TESTING ##########
year = 2018
table_class = 'stats_table'
df_columns = ['date', 'boxscore', 'team', 'at', 'opponent', 'w_or_l', 'runs', 'runs_allowed',
                'innings', 'record', 'div_rank', 'gb', 'winning_pitcher', 'losing_pitcher',
                'saving_pitcher', 'time', 'd_or_n', 'attendance', 'cli', 'streak', 'original_schedule']
url = 'https://www.baseball-reference.com/teams/NYM/2018-schedule-scores.shtml'
df = WebData(url).get_table_df(table_class, df_columns)
df.tail()

# processed_df = baseball_gamebygame_records_data_processing(df, year)
# processed_df.tail()

In [ ]:
########## baseball records ##########
def get_baseball_gamebygame_records():
  teams = ['NYN', 'CHN', 'ATL', 'CIN', 'FLO', 'MON', 'SLN', 'ANA', 'BAL',
         'MIN', 'OAK', 'TEX', 'TOR', 'ARI', 'PIT', 'SEA', 'HOU', 'KCA',
         'TBA', 'COL', 'MIL', 'PHI', 'SDN', 'SFN', 'BOS', 'DET', 'NYA',
         'LAN', 'CHA', 'CLE', 'WAS', 'MIA','LAA', 'WSN', 'KCR']
  year_range = range(2000, 2023)

  table_class = 'stats_table'
  df_columns = ['date', 'boxscore', 'team', 'at', 'opponent', 'w_or_l', 'runs', 'runs_allowed',
                'innings', 'record', 'div_rank', 'gb', 'winning_pitcher', 'losing_pitcher',
                'saving_pitcher', 'time', 'd_or_n', 'attendance', 'cli', 'streak', 'original_schedule']
  calculated_fields = ['season_end_rank',
                      'w_or_l_int', 'w_l_ratio', 'season_end_w_l_ratio', 'last_game_record',
                      'last_game_w_l_ratio', 'last_game_div_rank', 'last_game_gb',
                      'last_game_streak', 'runs_ma', 'runs_allowed_ma',
                      'season_end_runs_mean', 'season_end_runs_allowed_mean',
                      'last_ten_game_w_l_ratio']
  team_df = pd.DataFrame(columns=df_columns+calculated_fields)
  error_log = []

  for team in teams:
    for year in year_range:
      try:
        url = 'http://www.baseball-reference.com/teams/' + team + '/' + str(year) + '-schedule-scores.shtml'
        df = WebData(url).get_table_df(table_class, df_columns)

        # data processing, adding calculated fields
        df = baseball_gamebygame_records_data_processing(df, year)
        # filter down to only home games, avoid duplications
        df = df[~df['at'].str.contains('@')].reset_index(drop=True)
        # create one major df
        team_df = pd.concat([team_df, df]).reset_index(drop=True)
        # sleep
        sleep(4)

      except:
        error_log.append({'team':team, 'year':year})
        pass

  return team_df, error_log

In [ ]:
%time team_df, error_log = get_baseball_gamebygame_records()
team_df.to_csv('./data/mlb-game-data-2000-2022.csv', index=False, encoding='utf-8')

In [ ]:
baseball_reference.team.unique()

array(['ANA', 'ARI', 'ATL', 'BAL', 'BOS', 'CHC', 'CHW', 'CIN', 'CLE',
       'COL', 'DET', 'FLA', 'HOU', 'KCR', 'LAA', 'LAD', 'MIA', 'MIL',
       'MIN', 'MON', 'NYM', 'NYY', 'OAK', 'PHI', 'PIT', 'SDP', 'SEA',
       'SFG', 'STL', 'TBR', 'TEX', 'TOR', 'WSN'], dtype=object)

In [ ]:
########## nba schedule ##########
def get_nba_games():
    teams = ['BOS', 'CHI', 'DEN', 'MIN', 'DET', 'LAL', 'DAL', 'HOU', 
          'PHI', 'NYK', 'BRK', 'MIL', 'GSW', 'LAC', 'MIA', 'TOR',
          'WAS', 'PHO']
    year_range = range(2000, 2023)

    table_class = 'sortable stats_table'
    df_columns = ['date', 'start(ET)', 'null', 'box score', '@', 'opponent', 'w/l', 
                  'overtime', 'tm', 'opp', 'w', 'l', 'streak', 'notes']
    team_df = pd.DataFrame(columns=df_columns) 

    for team in teams:
      for year in year_range:
          try:
            url = 'http://www.basketball-reference.com/teams/' + team + '/' + str(year) + '_games.html'
            df = WebData(url).get_table_df(table_class, df_columns)
          # filter down to only home games, avoid duplications
            df = df[~df['@'].str.contains('@')].reset_index(drop=True)
          # create one major df
            team_df = pd.concat([team_df, df]).reset_index(drop=True)
          # sleep
            sleep(4)
          except:
              pass

    return team_df

In [ ]:
%time nba_records = get_nba_games()
# nba_records.to_csv('./data/nba-game-data-2000-2023.csv', index=False, encoding='utf-8')

In [ ]:
########## batting data ##########
def get_batting():
    team_df = pd.DataFrame(columns = ['Pos', 'Name', 'Age', 'G', 'PA', 'AB', 'R', 'H', '2B', '3B', 'HR',
                                      'RBI', 'SB', 'CS', 'BB', 'SO', 'BA', 'OBP', 'SLG', 'OPS', 'OPS+',
                                      'TB', 'GDP', 'HBP', 'SH', 'SF', 'IBB'])
    for team in teams:
        team_df = team_df.append(pd.Series(team, index=team_df.columns), ignore_index=True)
        for year in range (2000, 2023):
            try:
                team_df = team_df.append(pd.Series(year, index=team_df.columns), ignore_index=True)
                df = pd.DataFrame(columns = ['Pos', 'Name', 'Age', 'G', 'PA', 'AB', 'R', 'H', '2B', '3B', 'HR',
                                             'RBI', 'SB', 'CS', 'BB', 'SO', 'BA', 'OBP', 'SLG', 'OPS', 'OPS+',
                                             'TB', 'GDP', 'HBP', 'SH', 'SF', 'IBB'])
                url = 'http://www.baseball-reference.com/teams/' + team + '/' + str(year) + '.shtml'
                html = requests.get(url, proxies=proxy)
                bs = BeautifulSoup(html.text, 'html.parser')
                table = bs.find('table', {'id':'team_batting'})
                for game in table.find_all('tr'):
                    results = []
                    for element in game.find_all('td'):
                        results.append(element.text)
                        if len(results) == 27:
                            df = df.append(pd.Series(results, index=df.columns), ignore_index=True)

            # create one major df
                team_df = pd.concat([team_df, df]).reset_index(drop=True)

            # sleep
                sleep(3)
              
            except:
                pass

    return team_df


%time all_teams = get_batting()
all_teams.to_csv('./data/mlb-batting-data-2000-2022.csv', index=False, encoding='utf-8')

In [ ]:
########## pitching data ##########
def get_pitching():
    team_df = pd.DataFrame(columns = ['Pos', 'Name', 'Age', 'W', 'L', 'W-L%', 'ERA', 'G', 'GS', 'GF', 
                                      'CG', 'SHO', 'SV', 'IP', 'H', 'R', 'ER', 'HR', 'BB', 'IBB', 'SO',
                                      'HBP', 'BK', 'WP', 'BF', 'ERA+', 'FIP', 'WHIP', 'H9', 'HR9',
                                      'BB9', 'SO9', 'SO/W'])
    for team in teams:
        team_df = team_df.append(pd.Series(team, index=team_df.columns), ignore_index=True)
        for year in range (2000, 2023):
            try:
                team_df = team_df.append(pd.Series(year, index=team_df.columns), ignore_index=True)
                df = pd.DataFrame(columns = ['Pos', 'Name', 'Age', 'W', 'L', 'W-L%', 'ERA', 'G', 'GS', 'GF', 
                                      'CG', 'SHO', 'SV', 'IP', 'H', 'R', 'ER', 'HR', 'BB', 'IBB', 'SO',
                                      'HBP', 'BK', 'WP', 'BF', 'ERA+', 'FIP', 'WHIP', 'H9', 'HR9',
                                      'BB9', 'SO9', 'SO/W'])
                url = 'http://www.baseball-reference.com/teams/' + team + '/' + str(year) + '.shtml'
                html = requests.get(url, proxies=proxy)
                bs = BeautifulSoup(html.text, 'html.parser')
                table = bs.find('table', {'id':'team_pitching'})
                for game in table.find_all('tr'):
                    results = []
                    for element in game.find_all('td'):
                        results.append(element.text)
                        if len(results) == 33:
                            df = df.append(pd.Series(results, index=df.columns), ignore_index=True)

            # create one major df
                team_df = pd.concat([team_df, df]).reset_index(drop=True)

            # sleep
                sleep(3)
              
            except:
                pass

    return team_df


%time all_teams = get_pitching()
all_teams.to_csv('./data/mlb-pitching-data-2000-2022.csv', index=False, encoding='utf-8')